In [1]:
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.runnables import RunnablePassthrough, RunnableLambda, ConfigurableFieldSpec
# from langchain_core.output_parsers import StrOutputParser
# from langchain.memory import ConversationBufferWindowMemory
# from langchain.chains import create_history_aware_retriever, create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_community.chat_message_histories import ChatMessageHistory

In [1]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

chain = prompt | ChatOpenAI()

In [95]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec

chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda user_id, session_id: DynamoDBChatMessageHistory(
        table_name="lchain-ddb",
        session_id=session_id,
        key={"user_id": user_id, "session_id": session_id}
    ),
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ]
)

# This is where we configure the user and session id
config = {"configurable": {"user_id": "user_1", "session_id": "session_1"}}

In [96]:
# This is where we configure the user and session id
config = {"configurable": {"user_id": "user_1", "session_id": "session_1"}}

In [97]:
chain_with_history.invoke({"input": "Can you tell me my name?"}, config=config)

AIMessage(content='Pradhyumna', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 94, 'total_tokens': 99}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-56c70215-ced1-4015-a386-2f9c96990495-0', usage_metadata={'input_tokens': 94, 'output_tokens': 5, 'total_tokens': 99})

### Load and add data to the Vectorstore

In [31]:
from langchain_community.document_loaders import RecursiveUrlLoader
from bs4 import BeautifulSoup
import re

def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()


loader = RecursiveUrlLoader("https://python.langchain.com/v0.1/", extractor=bs4_extractor, max_depth=5)
all_docs = loader.load()
print(all_docs[0].page_content[:100])

d:\Applications\miniconda3\envs\ML\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Introduction | 🦜️🔗 LangChain

Skip to main contentLangChain v0.2 is out! You are currently viewing t


In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=100)
docs = splitter.transform_documents(all_docs)
print(f"Split documents into {len(docs)} chunks")

Split documents into 6252 chunks


In [16]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# vectorstore = PineconeVectorStore.from_documents(docs, embeddings, index_name="website-data")
vectorstore = PineconeVectorStore(index_name="car-data", embedding=embeddings) # Use this if you already have a Pinecone index
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
vectorstore.search("Your Query",search_type="similarity",k=10)

### Contextualization

In [17]:
from langchain.chains import create_history_aware_retriever

llm = ChatOpenAI(model="gpt-3.5-turbo", streaming=True)
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """
You are a Car Recommender chatbot. You are helping a user find a car that fits their needs.
The context provided to you is a mixture of car specs data as well as reviews and opinions on cars. Make sure you understand this before answering the user's question.
You should answer the question based only on the following context provided to you. If you don't have enough information to answer the question, you should say so.

Give the output in a nice markdown format.
Context:
{context}
####----####
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

#### Final Chain

In [19]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    lambda user_id, session_id: DynamoDBChatMessageHistory(
        table_name="lchain-ddb",
        session_id=session_id,
        key={"user_id": user_id, "session_id": session_id}
    ),
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
    history_factory_config=[
                    ConfigurableFieldSpec(
                        id="user_id",
                        annotation=str,
                        name="User ID",
                        description="Unique identifier for the user.",
                        default="",
                        is_shared=True,
                    ),
                    ConfigurableFieldSpec(
                        id="session_id",
                        annotation=str,
                        name="Conversation ID",
                        description="Unique identifier for the conversation.",
                        default="",
                        is_shared=True,
                    ),
                ],
)

In [22]:
config = {"configurable": {"user_id": "user2", "session_id": "session2"}}
conversational_rag_chain.invoke(
    {"input": "In Toyota?"},
    config=config, 
)["answer"]

'### Good SUVs in Toyota:\n\n1. **Toyota Urban Cruiser Hyryder:**\n   - Co-developed with Maruti, offering a well-rounded practical SUV.\n   - Features modern design, comfortable interior, and generous equipment list.\n   - Standout feature is the smooth hybrid technology.\n   - Lacks in performance compared to turbo-petrol rivals but excels in other areas.\n   \n2. **Toyota Fortuner:**\n   - Popular choice known for its eager performance and off-road capability.\n   - Considered the default choice in its segment with proven reliability.\n   - Offers broad, supportive, and comfortable seats with good ventilation.\n   - Not the most sophisticated SUV in terms of feel but valued for ownership ease and resale value.\n\nToyota offers SUVs that cater to different needs, with a focus on practicality, comfort, and reliability, making them popular choices in their respective segments.'

##### Streaming

In [ ]:
for answer in conversational_rag_chain.stream({"input": "In Toyota?"}, config=config):
        # Process and stream the output here
        for key in answer:
                if key == "answer":
                        print(answer['answer'], end="")